# Spoken language detection

## 1. 

In [2]:
import os
import random
import numpy as np
import tensorflow as tf
import librosa
import matplotlib.pyplot as plt
import pandas as pd
import IPython

In [85]:
PL_DIR = os.path.join('languages_audio', 'languages_audio_wav', 'pl', 'clips')
PL_DIR

'languages_audio\\languages_audio_wav\\pl\\clips'

In [86]:
# create tensorflow dataset
pl_dataset = tf.data.Dataset.list_files(PL_DIR + '\*.wav')
pl_dataset

<_ShuffleDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [87]:
pl_dataset.as_numpy_iterator().next()

b'languages_audio\\languages_audio_wav\\pl\\clips\\common_voice_pl_20701773.wav'

In [88]:
pl_dataset.as_numpy_iterator().next()

b'languages_audio\\languages_audio_wav\\pl\\clips\\common_voice_pl_20603034.wav'

In [89]:
df = pd.read_csv(os.path.join('languages_audio', 'languages_audio_mp3', 'pl', 'validated.tsv'), sep='\t')
df

C:\Users\Admin\AppData\Local\Temp\ipykernel_19928\1487738958.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join('languages_audio', 'languages_audio_mp3', 'pl', 'validated.tsv'), sep='\t')


,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,02eb087ff2dda32b8dd01a85b4693569a62d56033bf17d...,common_voice_pl_24005493.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,Ostatnie słowa wymówił syczącym szeptem i ze z...,NaN,2,0,NaN,NaN,NaN,NaN,pl,NaN
1,039674fdf440f2c9e53ff991cf27230071a2d969342dfd...,common_voice_pl_20605240.mp3,52f0b3164e79636bfc13b1cebe4c5665bb215bcf26095e...,Gdzież tu powód do wyrzutów?,NaN,5,0,NaN,NaN,NaN,NaN,pl,NaN
2,04bb285ec98789c6485651e8e7c5c5897de0462ce196bc...,common_voice_pl_20605185.mp3,5dd9590867924455c0b3cf4509c805d881a803371b385d...,Lubię go.,NaN,4,1,thirties,male_masculine,NaN,NaN,pl,NaN
3,068bf7cf01fb28afca5df71713cdbf9f52eb54e3d9eba2...,common_voice_pl_20633386.mp3,cc4d58bd50a90820cd2a7fe8ee771e35af924c61a56621...,— To mi nie pomaga.,NaN,3,1,NaN,NaN,NaN,NaN,pl,NaN
4,0935b1713f2c7947adace84ce007d74bf879f358e5c88d...,common_voice_pl_24105374.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,Ostatnie słowa wymówił syczącym szeptem i ze z...,NaN,2,0,NaN,NaN,NaN,NaN,pl,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131924,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659499.mp3,bf018139e9cb9c21570c757876f2fe4504f6c263d4b68b...,Porwała go i pobiegła do komory.,NaN,2,0,thirties,male_masculine,NaN,NaN,pl,NaN
131925,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659500.mp3,e355f502043dd8de0924144ad636a70466b17c01f7f897...,W drzwiach stał młody krępy górnik o szerokich...,NaN,2,0,thirties,male_masculine,NaN,NaN,pl,NaN
131926,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659501.mp3,3ad3ab45ec4017caf6945d45ef8106d91acbc7c7272a28...,"Naprawdę, nie wiedziałem, czy mam się cieszyć,...",NaN,2,0,thirties,male_masculine,NaN,NaN,pl,NaN
131927,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659502.mp3,0f436de5942a6681274c60800a6996d8fb4f28d4083bf8...,"Ściany były tak grube, że od sąsiedniego pokoj...",NaN,2,0,thirties,male_masculine,NaN,NaN,pl,NaN


In [90]:
df[df['gender'] == 'female_feminine']

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
24,26b6fc36bff0448de0ed25e2b660056ad036f96005bda9...,common_voice_pl_23694146.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,Ostatnie słowa wymówił syczącym szeptem i ze z...,NaN,2,0,twenties,female_feminine,NaN,NaN,pl,NaN
158,cdd9e3173cd44c3b6695144072c4cf45b7a24a795def30...,common_voice_pl_23579454.mp3,91b75d2edf9c5e09c06eb9191f8f5d3fc97bf0f5a21e37...,Mnie to interesuje.,NaN,2,0,twenties,female_feminine,NaN,NaN,pl,NaN
311,8b6747c24dfbc22ebb50658b742d9e4e6dd59f2d412636...,common_voice_pl_27100225.mp3,0a17d2f0b55ad6227f51534fcc97664417a192e027980a...,Brakuje nam czasu,NaN,2,0,twenties,female_feminine,NaN,NaN,pl,NaN
312,8b6747c24dfbc22ebb50658b742d9e4e6dd59f2d412636...,common_voice_pl_27100226.mp3,0a4f7c81091a523aa2ef361c6bf3f56d45f73cfe16483e...,"Nie ma wątpliwości, że klimat się zmienia",NaN,2,1,twenties,female_feminine,NaN,NaN,pl,NaN
691,c0af0a34f01c2dd7bf4905cae03217ccc228312b8763ea...,common_voice_pl_39597727.mp3,1da9e571e1ab62bb331baf7fea3b07341b281c0782c5d3...,Zmiana wspólnej polityki rolnej,NaN,2,0,seventies,female_feminine,NaN,NaN,pl,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115900,d9c913215277238571f601e0e7963bb1dc371ec6e353b5...,common_voice_pl_26587805.mp3,0958ff79a549e7528519b39cdfca5ae257aa3c90d11be5...,uchylanie się od opodatkowania,NaN,2,0,twenties,female_feminine,NaN,NaN,pl,NaN
115901,d9c913215277238571f601e0e7963bb1dc371ec6e353b5...,common_voice_pl_26587807.mp3,0930c019b37b5ba851c5c0f77d84694a59d8fae415e218...,"Uważam również, że debata ta pokazała, jak wie...",NaN,2,0,twenties,female_feminine,NaN,NaN,pl,NaN
115902,d9c913215277238571f601e0e7963bb1dc371ec6e353b5...,common_voice_pl_26587809.mp3,093caf621077839d205578607762a57b48773fac57ee84...,I popełnia się ją pod obłudną przykrywką rzeko...,NaN,2,0,twenties,female_feminine,NaN,NaN,pl,NaN
115903,d9c913215277238571f601e0e7963bb1dc371ec6e353b5...,common_voice_pl_26587813.mp3,09a6ab44f2bcd21e58ba65e1d9051f75fd6fb24d0cf384...,Wydaje się to technicznie możliwe i zostało ju...,NaN,2,0,twenties,female_feminine,NaN,NaN,pl,NaN


In [91]:
df[df['gender'] == 'male_masculine']

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
2,04bb285ec98789c6485651e8e7c5c5897de0462ce196bc...,common_voice_pl_20605185.mp3,5dd9590867924455c0b3cf4509c805d881a803371b385d...,Lubię go.,NaN,4,1,thirties,male_masculine,NaN,NaN,pl,NaN
22,2486bf621942aa6e36ab658e7791aa38563bf908c2485c...,common_voice_pl_23840821.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,Ostatnie słowa wymówił syczącym szeptem i ze z...,NaN,2,0,thirties,male_masculine,NaN,NaN,pl,NaN
37,3358c4d8d0cbcdaed3828313b13ad54cb185fa9bbe17a0...,common_voice_pl_24085740.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,Ostatnie słowa wymówił syczącym szeptem i ze z...,NaN,2,0,twenties,male_masculine,NaN,NaN,pl,NaN
49,46ae7820b8db907ba0951c0db2aca13e4061be363f5b67...,common_voice_pl_23677778.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,Ostatnie słowa wymówił syczącym szeptem i ze z...,NaN,2,0,twenties,male_masculine,NaN,NaN,pl,NaN
52,4d3b0e719909ca39f14a72a0c2af34145bb10442bd1cbc...,common_voice_pl_20605456.mp3,3369c1ddb7fb5a2ecf2c155634718817a170f3f8e613ef...,"Ona, która go zrozumiała zbyt wąsko: — Ooo, tak.",NaN,7,0,thirties,male_masculine,NaN,NaN,pl,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131924,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659499.mp3,bf018139e9cb9c21570c757876f2fe4504f6c263d4b68b...,Porwała go i pobiegła do komory.,NaN,2,0,thirties,male_masculine,NaN,NaN,pl,NaN
131925,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659500.mp3,e355f502043dd8de0924144ad636a70466b17c01f7f897...,W drzwiach stał młody krępy górnik o szerokich...,NaN,2,0,thirties,male_masculine,NaN,NaN,pl,NaN
131926,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659501.mp3,3ad3ab45ec4017caf6945d45ef8106d91acbc7c7272a28...,"Naprawdę, nie wiedziałem, czy mam się cieszyć,...",NaN,2,0,thirties,male_masculine,NaN,NaN,pl,NaN
131927,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659502.mp3,0f436de5942a6681274c60800a6996d8fb4f28d4083bf8...,"Ściany były tak grube, że od sąsiedniego pokoj...",NaN,2,0,thirties,male_masculine,NaN,NaN,pl,NaN


In [92]:
df.groupby('sentence_id').count()

,client_id,path,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
sentence_id,,,,,,,,,,,,
000010547254f2aa77c475053f8b6b4c83d434f9bc6b39febb275b766c0e4cb7,1,1,1,0,1,1,1,1,1,0,1,0
00007f59e910334724585497c5f56c66445dad9a96bad62108918f507c46d70a,1,1,1,0,1,1,1,1,0,0,1,0
0000a42a2e6b7578771b68c98e11c74b5b086a2ac027592edfeb99e97c6dbf79,1,1,1,0,1,1,1,1,0,0,1,0
00025f331e89b18ae83e113ca0b1809e73ebb3835b9d7bebdd9db4e8f0a35a83,1,1,1,0,1,1,1,1,0,0,1,0
00026e9700c3b59e381a73ca0363782efe5016b2375c5693af2e40f80face7fb,1,1,1,0,1,1,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
ffecbc28db5bb94420e0417acfd68a3eeaa44c473e0e55ebb626c3ab3df4bb23,3,3,3,0,3,3,3,3,0,0,3,0
ffef45483b2128ee9ddde50f72279d72bd4122b51298735fa6c8ec060356abfd,2,2,2,0,2,2,2,2,0,0,2,0
fff04179750ea84cc1838064e42ab2b595479bec71425ae00e5ea1fd21feb01b,3,3,3,0,3,3,3,3,0,0,3,0


In [93]:
df['sentence_id'].value_counts()

sentence_id
4d7eae22f3184a3b7a662defeceec08aa69b68109ede2e5d699534f545214e98    84
4c97d8f9de17694342d66084c820814bfa5df996855bfffedee0dc03f523a486    84
50ad0cc3186e9995592d55fcd0cd4996cd5952695a1206ccf9ef11069501fe14    84
c52308905f72519ecbbe14b64284126112d1e28732894654f71bc4493fa2ab4b    83
60a5f73c90d77d1fe0f2a660c6bbbadc679ece26a0f464691e319c65c5f5a1ed    83
                                                                    ..
15d001c97690da2440328a6113baaaf55b30ccb3ae9edadf7ef2364c9daf5340     1
1685bd829c38150f7547a850121a61d9aac17d99e838e1dfe251aab05539f3e0     1
166208c753de3c85dc7f9907fd9a4281c70383509cb22d881556ba2933fad062     1
1683b06e44813c337584ce1fadf69629488b57c4990d3d133921ec3d16ab0163     1
710a2e0d24b29da1427629d278732074f4b39b202e52b17ef7299c09b47670e0     1
Name: count, Length: 38536, dtype: int64

In [94]:
df['sentence_id'].nunique()

38536

In [95]:
df['client_id'].value_counts().sort_values(ascending=True)

client_id
02eb087ff2dda32b8dd01a85b4693569a62d56033bf17d7f0565aa45fa29b850e53f808baf94fef3d89be8a5ed0432675594297e47f8e5ea0ddfe86e81a21711        1
068bf7cf01fb28afca5df71713cdbf9f52eb54e3d9eba2bfc971b17cd2aac103a9ef9d74aa63e6471f6562e10320a959623fc0051de88c1cc7662ecffa5b768b        1
0935b1713f2c7947adace84ce007d74bf879f358e5c88df2648d8c0f5176bbc127658efce31ec55938ba30f7359eef9d9eeca55082328732e855e21927e74ff0        1
09e8fc1e934239cfd3124d1ca7f1c4f3d27cde45951bb4a0d8b0014c879f46139067292d9eb3e47b9f210f3207f858add3100b8cdf8df7eaee1b1b6269de0f55        1
0b57062fe0a2d9f1996677fa64a27c01d528f83ab99a419324b4f46584c114374ab44c4df19d8d5c8e9fa7c383cb4d78af384a9ba0bd01af3b4e3d9f0041e303        1
                                                                                                                                    ...  
2c818dfafa7197d291cac63a8b8f41377c82ebf481d2a81244a0329a579d58a91d57d2de0938abb8b4dda68239b41046591191bf1400b012becb05434b37bb36     3484
b1fc1ae975d3f14e8d7a508d

In [96]:
df[['sentence_id', 'client_id']].groupby(by='sentence_id').count().sort_values(by='client_id', ascending=False)


,client_id
sentence_id,
4c97d8f9de17694342d66084c820814bfa5df996855bfffedee0dc03f523a486,84
50ad0cc3186e9995592d55fcd0cd4996cd5952695a1206ccf9ef11069501fe14,84
4d7eae22f3184a3b7a662defeceec08aa69b68109ede2e5d699534f545214e98,84
f2505897bb27ca76f4b45039caf56ccab070b95f0737bd3211845ea1881b0eff,83
60a5f73c90d77d1fe0f2a660c6bbbadc679ece26a0f464691e319c65c5f5a1ed,83
...,...
135d28e4140fed1824b11f9571e100f35c5094f6164fc6cf548ce8b978b90e4b,1
135cfc46346937a25472447586b8aef9e8a40277e1bfece3aa6efe310fb4cdd3,1
135cb6eddb5ccb7a713f55e8f1f5732c1612b49508c066c36c8660bd416beb5b,1


In [97]:
df[['client_id', 'sentence_id']].sort_values(by='client_id', ascending=False)

,client_id,sentence_id
6429,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,c8d122e3a38c2406cfc98e1c92c9b6bfa42360a01ca634...
6431,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,7c049b380c12bde413ab3eb2579079914cb0307db26e8c...
6430,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,b7f580c9ac8f2c2c8e20285a0c16521ab3a979f233277d...
6427,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,ccb6b126fe4d1161924fbc663271fc5fda7308aeee1ceb...
6428,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,c7727ea0625ec2b75629ce52d623e562ff65873c376dc2...
...,...,...
2182,000266163565c94f08a466f947dfae7645d83ee9705ee8...,e7f4a9286200d2cf390df58b3258e982c413ec01dc55e3...
2183,000266163565c94f08a466f947dfae7645d83ee9705ee8...,dfd693f167678a92f8e57fb22361b93d47d2fac99b7f7a...
2184,000266163565c94f08a466f947dfae7645d83ee9705ee8...,edda6b80d957c47119f35d79a2cd2e0b2cdf7ab9e0d332...
2185,000266163565c94f08a466f947dfae7645d83ee9705ee8...,f067783bad53e3b3234e06413876aaf070b4896434efd3...


In [98]:
df[['client_id', 'sentence_id']].sort_values(by='client_id', ascending=False)

,client_id,sentence_id
6429,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,c8d122e3a38c2406cfc98e1c92c9b6bfa42360a01ca634...
6431,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,7c049b380c12bde413ab3eb2579079914cb0307db26e8c...
6430,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,b7f580c9ac8f2c2c8e20285a0c16521ab3a979f233277d...
6427,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,ccb6b126fe4d1161924fbc663271fc5fda7308aeee1ceb...
6428,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,c7727ea0625ec2b75629ce52d623e562ff65873c376dc2...
...,...,...
2182,000266163565c94f08a466f947dfae7645d83ee9705ee8...,e7f4a9286200d2cf390df58b3258e982c413ec01dc55e3...
2183,000266163565c94f08a466f947dfae7645d83ee9705ee8...,dfd693f167678a92f8e57fb22361b93d47d2fac99b7f7a...
2184,000266163565c94f08a466f947dfae7645d83ee9705ee8...,edda6b80d957c47119f35d79a2cd2e0b2cdf7ab9e0d332...
2185,000266163565c94f08a466f947dfae7645d83ee9705ee8...,f067783bad53e3b3234e06413876aaf070b4896434efd3...


In [99]:
df[df['client_id'] == 'ffffc00a5ed69f08a486837064ec2caeff21fe06264c3dd733f633fb6c2ae9aeb561a5a6f43e000554d4b3cc171644ba4ce971177af1cb54f9bd8cc153e71a5c']

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
6427,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,common_voice_pl_20642313.mp3,ccb6b126fe4d1161924fbc663271fc5fda7308aeee1ceb...,"Nie znam takiego, a jednak widziałem go nagieg...",NaN,2,0,NaN,NaN,NaN,NaN,pl,NaN
6428,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,common_voice_pl_20642315.mp3,c7727ea0625ec2b75629ce52d623e562ff65873c376dc2...,"Apollo, chcąc objąć Daphne — objął krzew laurowy.",NaN,3,0,NaN,NaN,NaN,NaN,pl,NaN
6429,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,common_voice_pl_20642317.mp3,c8d122e3a38c2406cfc98e1c92c9b6bfa42360a01ca634...,I mało w tym było pozy.,NaN,2,0,NaN,NaN,NaN,NaN,pl,NaN
6430,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,common_voice_pl_20642319.mp3,b7f580c9ac8f2c2c8e20285a0c16521ab3a979f233277d...,Urządzono w wielu miastach kluby dla dzieci.,NaN,2,0,NaN,NaN,NaN,NaN,pl,NaN
6431,ffffc00a5ed69f08a486837064ec2caeff21fe06264c3d...,common_voice_pl_20642321.mp3,7c049b380c12bde413ab3eb2579079914cb0307db26e8c...,Pewnego razu ona powiedziała do niego: — Twoje...,NaN,2,0,NaN,NaN,NaN,NaN,pl,NaN


In [100]:
# # sprawdzenie czy client_id oraz sequence_id się nie duplikują (czy jeden klient ma rózne sekwencej)
# 
# for client_id in df['client_id'].unique():
#     sentence_count = df['sentence_id'][df['client_id']  == client_id].count()
#     client_id_count = df['client_id'][df['client_id']  == client_id].value_counts().iloc[0]
#     if sentence_count != client_id_count:
#         print(True)


In [101]:
df['sentence_id'][df['client_id']  == 'ffffc00a5ed69f08a486837064ec2caeff21fe06264c3dd733f633fb6c2ae9aeb561a5a6f43e000554d4b3cc171644ba4ce971177af1cb54f9bd8cc153e71a5c'].count()

5

In [102]:
df['client_id'][df['client_id']  == 'ffffc00a5ed69f08a486837064ec2caeff21fe06264c3dd733f633fb6c2ae9aeb561a5a6f43e000554d4b3cc171644ba4ce971177af1cb54f9bd8cc153e71a5c'].value_counts()

client_id
ffffc00a5ed69f08a486837064ec2caeff21fe06264c3dd733f633fb6c2ae9aeb561a5a6f43e000554d4b3cc171644ba4ce971177af1cb54f9bd8cc153e71a5c    5
Name: count, dtype: int64

In [103]:
# TODO: WZiąć plik validated.tsv nastepnie podzielić go na mężczyzn i kobiety (dwa osobne dataset). następnie wrzucać do każdego ze zbiorów (najpiew testwoy, walidacyjny, treningowy) client_id od najmniejszej liczby wystąpień w obydwu  dopasować czas trwania każdej próbki (najpierw obcinać od środka, potem dopełniać zerami), zmienić na spektogram i dopiero wtedy dodać (koniec funkcji). druga funkcja będzie jako augumentowanie losowych danych (losowo wybiera funkcję która coś zrobi z danymi) jeżeli moje zbiory kobiet i mężczyzn nie będą spełniały wymaganej ilości próbek którą się poda. Na wyjściu ma być tensor z próbkami spektogramu. To zrobić finalnie dla każdego folderu (języka). Wtedy dopiero dodać label w formie one hot encoding

## Load data:

In [12]:
df = pd.read_csv(os.path.join('..', 'pl', 'validated.tsv'), sep='\t', usecols=['client_id', 'path', 'sentence_id', 'gender', 'locale'])

## Change mp3 to wav name

In [13]:
# df = df.apply(lambda path: path.str.replace('.mp3', '.wav'))

## Split to woman and man sets

In [14]:
df

,client_id,path,sentence_id,gender,locale
0,02eb087ff2dda32b8dd01a85b4693569a62d56033bf17d...,common_voice_pl_24005493.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,NaN,pl
1,039674fdf440f2c9e53ff991cf27230071a2d969342dfd...,common_voice_pl_20605240.mp3,52f0b3164e79636bfc13b1cebe4c5665bb215bcf26095e...,NaN,pl
2,04bb285ec98789c6485651e8e7c5c5897de0462ce196bc...,common_voice_pl_20605185.mp3,5dd9590867924455c0b3cf4509c805d881a803371b385d...,male_masculine,pl
3,068bf7cf01fb28afca5df71713cdbf9f52eb54e3d9eba2...,common_voice_pl_20633386.mp3,cc4d58bd50a90820cd2a7fe8ee771e35af924c61a56621...,NaN,pl
4,0935b1713f2c7947adace84ce007d74bf879f358e5c88d...,common_voice_pl_24105374.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,NaN,pl
...,...,...,...,...,...
131924,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659499.mp3,bf018139e9cb9c21570c757876f2fe4504f6c263d4b68b...,male_masculine,pl
131925,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659500.mp3,e355f502043dd8de0924144ad636a70466b17c01f7f897...,male_masculine,pl
131926,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659501.mp3,3ad3ab45ec4017caf6945d45ef8106d91acbc7c7272a28...,male_masculine,pl
131927,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659502.mp3,0f436de5942a6681274c60800a6996d8fb4f28d4083bf8...,male_masculine,pl


In [15]:
woman_filter = df['gender'] == 'female_feminine'
man_filter = df['gender'] == 'male_masculine'

In [16]:
df_women = df[woman_filter]
df_women

,client_id,path,sentence_id,gender,locale
24,26b6fc36bff0448de0ed25e2b660056ad036f96005bda9...,common_voice_pl_23694146.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,female_feminine,pl
158,cdd9e3173cd44c3b6695144072c4cf45b7a24a795def30...,common_voice_pl_23579454.mp3,91b75d2edf9c5e09c06eb9191f8f5d3fc97bf0f5a21e37...,female_feminine,pl
311,8b6747c24dfbc22ebb50658b742d9e4e6dd59f2d412636...,common_voice_pl_27100225.mp3,0a17d2f0b55ad6227f51534fcc97664417a192e027980a...,female_feminine,pl
312,8b6747c24dfbc22ebb50658b742d9e4e6dd59f2d412636...,common_voice_pl_27100226.mp3,0a4f7c81091a523aa2ef361c6bf3f56d45f73cfe16483e...,female_feminine,pl
691,c0af0a34f01c2dd7bf4905cae03217ccc228312b8763ea...,common_voice_pl_39597727.mp3,1da9e571e1ab62bb331baf7fea3b07341b281c0782c5d3...,female_feminine,pl
...,...,...,...,...,...
115900,d9c913215277238571f601e0e7963bb1dc371ec6e353b5...,common_voice_pl_26587805.mp3,0958ff79a549e7528519b39cdfca5ae257aa3c90d11be5...,female_feminine,pl
115901,d9c913215277238571f601e0e7963bb1dc371ec6e353b5...,common_voice_pl_26587807.mp3,0930c019b37b5ba851c5c0f77d84694a59d8fae415e218...,female_feminine,pl
115902,d9c913215277238571f601e0e7963bb1dc371ec6e353b5...,common_voice_pl_26587809.mp3,093caf621077839d205578607762a57b48773fac57ee84...,female_feminine,pl
115903,d9c913215277238571f601e0e7963bb1dc371ec6e353b5...,common_voice_pl_26587813.mp3,09a6ab44f2bcd21e58ba65e1d9051f75fd6fb24d0cf384...,female_feminine,pl


In [17]:
df_men = df[man_filter]
df_men

,client_id,path,sentence_id,gender,locale
2,04bb285ec98789c6485651e8e7c5c5897de0462ce196bc...,common_voice_pl_20605185.mp3,5dd9590867924455c0b3cf4509c805d881a803371b385d...,male_masculine,pl
22,2486bf621942aa6e36ab658e7791aa38563bf908c2485c...,common_voice_pl_23840821.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,male_masculine,pl
37,3358c4d8d0cbcdaed3828313b13ad54cb185fa9bbe17a0...,common_voice_pl_24085740.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,male_masculine,pl
49,46ae7820b8db907ba0951c0db2aca13e4061be363f5b67...,common_voice_pl_23677778.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,male_masculine,pl
52,4d3b0e719909ca39f14a72a0c2af34145bb10442bd1cbc...,common_voice_pl_20605456.mp3,3369c1ddb7fb5a2ecf2c155634718817a170f3f8e613ef...,male_masculine,pl
...,...,...,...,...,...
131924,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659499.mp3,bf018139e9cb9c21570c757876f2fe4504f6c263d4b68b...,male_masculine,pl
131925,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659500.mp3,e355f502043dd8de0924144ad636a70466b17c01f7f897...,male_masculine,pl
131926,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659501.mp3,3ad3ab45ec4017caf6945d45ef8106d91acbc7c7272a28...,male_masculine,pl
131927,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659502.mp3,0f436de5942a6681274c60800a6996d8fb4f28d4083bf8...,male_masculine,pl


## Set number of probes in my set and split it to train, validate and test

In [56]:
SET_SIZE = 3_000
MAX_NUMBER_OF_CLIENT_ID = 3000
MIN_CLIP_DURATION = 4000 #  clip time in [ms]

TRAIN_SIZE = int(SET_SIZE * 0.6)
VAL_SIZE = int((SET_SIZE - TRAIN_SIZE) // 2)
TEST_SIZE = SET_SIZE - TRAIN_SIZE - VAL_SIZE

print(f'Train size: {TRAIN_SIZE}')
print(f'Validation size: {VAL_SIZE}')
print(f'Test size: {TEST_SIZE}')
print(f'Sum of sizes: {TRAIN_SIZE + VAL_SIZE + TEST_SIZE} is equal to SET_SIZE {TRAIN_SIZE + VAL_SIZE + TEST_SIZE == SET_SIZE}')

Train size: 1800
Validation size: 600
Test size: 600
Sum of sizes: 3000 is equal to SET_SIZE True


In [57]:
# Select probes with min that time 
df_clips_duration = pd.read_csv(os.path.join('..', 'pl', 'clip_durations.tsv'), sep='\t')
df_clips_duration['clip'] = df_clips_duration['clip']
df_men = df_men.merge(df_clips_duration, left_on='path', right_on='clip')
df_men

,client_id,path,sentence_id,gender,locale,clip_x,duration[ms]_x,clip_y,duration[ms]_y
0,04bb285ec98789c6485651e8e7c5c5897de0462ce196bc...,common_voice_pl_20605185.mp3,5dd9590867924455c0b3cf4509c805d881a803371b385d...,male_masculine,pl,common_voice_pl_20605185.mp3,7584,common_voice_pl_20605185.mp3,7584
1,2486bf621942aa6e36ab658e7791aa38563bf908c2485c...,common_voice_pl_23840821.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,male_masculine,pl,common_voice_pl_23840821.mp3,7944,common_voice_pl_23840821.mp3,7944
2,3358c4d8d0cbcdaed3828313b13ad54cb185fa9bbe17a0...,common_voice_pl_24085740.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,male_masculine,pl,common_voice_pl_24085740.mp3,7488,common_voice_pl_24085740.mp3,7488
3,46ae7820b8db907ba0951c0db2aca13e4061be363f5b67...,common_voice_pl_23677778.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,male_masculine,pl,common_voice_pl_23677778.mp3,5256,common_voice_pl_23677778.mp3,5256
4,4d3b0e719909ca39f14a72a0c2af34145bb10442bd1cbc...,common_voice_pl_20605456.mp3,3369c1ddb7fb5a2ecf2c155634718817a170f3f8e613ef...,male_masculine,pl,common_voice_pl_20605456.mp3,6456,common_voice_pl_20605456.mp3,6456
...,...,...,...,...,...,...,...,...,...
42463,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659484.mp3,a83ee5b0dfc776d8dfb29934fd4b9410fd1fd1d1e780c7...,male_masculine,pl,common_voice_pl_21659484.mp3,4656,common_voice_pl_21659484.mp3,4656
42464,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659500.mp3,e355f502043dd8de0924144ad636a70466b17c01f7f897...,male_masculine,pl,common_voice_pl_21659500.mp3,6696,common_voice_pl_21659500.mp3,6696
42465,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659501.mp3,3ad3ab45ec4017caf6945d45ef8106d91acbc7c7272a28...,male_masculine,pl,common_voice_pl_21659501.mp3,5304,common_voice_pl_21659501.mp3,5304
42466,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659502.mp3,0f436de5942a6681274c60800a6996d8fb4f28d4083bf8...,male_masculine,pl,common_voice_pl_21659502.mp3,6024,common_voice_pl_21659502.mp3,6024


In [58]:

# 
# for i in range(len(df_clips_duration)):
#     print(i)
#     row_in_df = df_men[df_men['path']== df_clips_duration.iloc[i]['clip']] 
#     if (df_clips_duration.iloc[i]['duration[ms]'] >= MIN_CLIP_DURATION) and (len(row_in_df) >= 0):
#         rows_over_min_dur_time = pd.concat([rows_over_min_dur_time, row_in_df])
#     else:
#         rows_under_min_dur_time = pd.concat([rows_under_min_dur_time, row_in_df])
# 
# 

rows_over_min_dur_time = df_men[df_men['duration[ms]'] >= MIN_CLIP_DURATION]
rows_under_min_dur_time = df_men[df_men['duration[ms]'] < MIN_CLIP_DURATION]
rows_over_min_dur_time

KeyError: 'duration[ms]'

In [59]:
rows_under_min_dur_time

,client_id,path,sentence_id,gender,locale,clip,duration[ms]
9,9d9a5ec6ddce58dc304abcc92d2ad273c8c2736cf253d6...,common_voice_pl_20605516.mp3,3f2606624aca7b8a9d8ec80f586460a095d12ebf5e6f08...,male_masculine,pl,common_voice_pl_20605516.mp3,3216
21,3c3970d04fe7c7271d9e6b1426e00beb1b51a977fe4787...,common_voice_pl_20605922.mp3,780aaee039d89108aae855d281233b7975615cc3a1eb4a...,male_masculine,pl,common_voice_pl_20605922.mp3,3624
25,7faf1b2a8074d32814fd8c78aaaea5abe0aa23ca45f183...,common_voice_pl_22723318.mp3,1e30e5a51b1d04b6011b2c76995ab608d0dd0a59611d84...,male_masculine,pl,common_voice_pl_22723318.mp3,3048
28,da9b81e6715e1dec2a6ab9b92e35768840b763b290feaf...,common_voice_pl_20613227.mp3,63b0704f37221e7cdde32b87a84d68d8005b4d73e22704...,male_masculine,pl,common_voice_pl_20613227.mp3,3624
30,e09d32c81be499447e2a638ed932c0bad5c3c89820072c...,common_voice_pl_20703091.mp3,893fa0aa1d9c33e69338f61d425a1e6c46a9158f1906f4...,male_masculine,pl,common_voice_pl_20703091.mp3,2952
...,...,...,...,...,...,...,...
79861,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659478.mp3,b475a12d8bc4a52709050a96f96dbfe27684665088734e...,male_masculine,pl,common_voice_pl_21659478.mp3,2304
79863,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659481.mp3,d2972df9aa18c9440e1324c88d5c0f7d4e0f3aa571f368...,male_masculine,pl,common_voice_pl_21659481.mp3,3024
79864,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659482.mp3,e8d211592601ff9cdf7709620533c94a8a12dce393a085...,male_masculine,pl,common_voice_pl_21659482.mp3,3024
79866,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659485.mp3,c52aba9d343f2d9235404e9e60161ecd4255ef218b21ef...,male_masculine,pl,common_voice_pl_21659485.mp3,3936


In [60]:
if len(rows_over_min_dur_time) >= SET_SIZE:
    df_men = rows_over_min_dur_time
else:
    df_men = pd.concat([rows_over_min_dur_time, rows_under_min_dur_time], ignore_index=True)[:SET_SIZE]

df_men

,client_id,path,sentence_id,gender,locale,clip,duration[ms]
0,04bb285ec98789c6485651e8e7c5c5897de0462ce196bc...,common_voice_pl_20605185.mp3,5dd9590867924455c0b3cf4509c805d881a803371b385d...,male_masculine,pl,common_voice_pl_20605185.mp3,7584
1,2486bf621942aa6e36ab658e7791aa38563bf908c2485c...,common_voice_pl_23840821.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,male_masculine,pl,common_voice_pl_23840821.mp3,7944
2,3358c4d8d0cbcdaed3828313b13ad54cb185fa9bbe17a0...,common_voice_pl_24085740.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,male_masculine,pl,common_voice_pl_24085740.mp3,7488
3,46ae7820b8db907ba0951c0db2aca13e4061be363f5b67...,common_voice_pl_23677778.mp3,3538b2608e1451fa8cb7aea203d1515df86d6a4f933bde...,male_masculine,pl,common_voice_pl_23677778.mp3,5256
4,4d3b0e719909ca39f14a72a0c2af34145bb10442bd1cbc...,common_voice_pl_20605456.mp3,3369c1ddb7fb5a2ecf2c155634718817a170f3f8e613ef...,male_masculine,pl,common_voice_pl_20605456.mp3,6456
...,...,...,...,...,...,...,...
79865,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659484.mp3,a83ee5b0dfc776d8dfb29934fd4b9410fd1fd1d1e780c7...,male_masculine,pl,common_voice_pl_21659484.mp3,4656
79868,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659500.mp3,e355f502043dd8de0924144ad636a70466b17c01f7f897...,male_masculine,pl,common_voice_pl_21659500.mp3,6696
79869,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659501.mp3,3ad3ab45ec4017caf6945d45ef8106d91acbc7c7272a28...,male_masculine,pl,common_voice_pl_21659501.mp3,5304
79870,8c4eeac4b0268f6da0712d963ed4c527e004098ca234ba...,common_voice_pl_21659502.mp3,0f436de5942a6681274c60800a6996d8fb4f28d4083bf8...,male_masculine,pl,common_voice_pl_21659502.mp3,6024


In [61]:
df_train = pd.DataFrame()
df_val = pd.DataFrame()
df_test = pd.DataFrame()

In [62]:
df_counted_id = df_men['client_id'].value_counts(ascending=True)
df_counted_id

client_id
04bb285ec98789c6485651e8e7c5c5897de0462ce196bc909dd1f465102a2e871f97caac708eab4ef1a226eeafccbdf4e5f801557e6f78e3e7ae890695341e0f       1
e6fed530cba7542c8204d127f2209550e91225077b7942fab61b349aba316e815d7fd75e44024cd6b1d0ba55642bb74a3748250622e9f2478a37a133b7093acb       1
f4615d22c77c3b141876c3b85ce3ddb40f4aac53c17097d5be1f6b63968bb79c49f832c956df3eb364a3d7ed9b39334d915f96119ebffc6cbba618f398564130       1
b3e1520c45975f3e529861c29465ef0fc045dce3690b4ec1feb9214cb4d6b40b55df6f09180179b5770585d58babc04c346c5c60adb9bce8ce2a4066adc120d5       1
09b75889161f87c06d50d3620996eef275cdcb06ffd2eedfd869b6da99884dd38c31feee2180e14bb6485410808d05c0efafcae9460e6cd22cf966d5103ef85b       1
                                                                                                                                    ... 
05beeef7c41d60290389ab0dd20989132ed1f33241f542fdad335ef3aa93e8b00863a81be978590d54a57193c57166cab306af7ad7114e30b37f18bfa20cdee3    1391
6f2dd557da863cb4e0303da6ca2b1ab

In [63]:
clients_form_origin_df = df_men[df_men['client_id'] == df_counted_id.index[200]]
clients_form_origin_df1 = df_men[df_men['client_id'] == df_counted_id.index[201]]
clients_form_origin_df1['client_id'].iloc[0]

'59735530ed153d5ce3a66ef9103f401a00b880b7febb191db3725d17a93b9fd0275e9e0452be57eadce9de682d48b7d1582dedcb337a859dbda69f12facb96ed'

In [64]:
for i in range(len(df_counted_id)):
    print(df_counted_id.index[i], df_counted_id.iloc[i])
    rows_form_origin_df = df_men[df_men['client_id'] == df_counted_id.index[i]][:MAX_NUMBER_OF_CLIENT_ID]
    if len(rows_form_origin_df) <= (TEST_SIZE - len(df_test)) and rows_form_origin_df['client_id'].iloc[0]:
        df_test = pd.concat([df_test, rows_form_origin_df], ignore_index=True)
        continue
    if len(df_test) < TEST_SIZE and rows_form_origin_df['client_id'].iloc[0]:
        df_test = pd.concat([df_test, rows_form_origin_df[:(TEST_SIZE - len(df_test))]], ignore_index=True)
        continue
    if len(rows_form_origin_df) <= (VAL_SIZE - len(df_val)):
        df_val = pd.concat([df_val, rows_form_origin_df], ignore_index=True)
        continue
    if len(df_val) < VAL_SIZE and rows_form_origin_df['client_id'].iloc[0]:
        df_val = pd.concat([df_val, rows_form_origin_df[:(VAL_SIZE - len(df_val))]], ignore_index=True)
        continue
    if len(rows_form_origin_df) <= (TRAIN_SIZE - len(df_train)):
        df_train = pd.concat([df_train, rows_form_origin_df], ignore_index=True)
        continue
    

04bb285ec98789c6485651e8e7c5c5897de0462ce196bc909dd1f465102a2e871f97caac708eab4ef1a226eeafccbdf4e5f801557e6f78e3e7ae890695341e0f 1
e6fed530cba7542c8204d127f2209550e91225077b7942fab61b349aba316e815d7fd75e44024cd6b1d0ba55642bb74a3748250622e9f2478a37a133b7093acb 1
f4615d22c77c3b141876c3b85ce3ddb40f4aac53c17097d5be1f6b63968bb79c49f832c956df3eb364a3d7ed9b39334d915f96119ebffc6cbba618f398564130 1
b3e1520c45975f3e529861c29465ef0fc045dce3690b4ec1feb9214cb4d6b40b55df6f09180179b5770585d58babc04c346c5c60adb9bce8ce2a4066adc120d5 1
09b75889161f87c06d50d3620996eef275cdcb06ffd2eedfd869b6da99884dd38c31feee2180e14bb6485410808d05c0efafcae9460e6cd22cf966d5103ef85b 1
aec84a44ce3730faa1d1893783b6f2c23b8de7d31b68d6833a16b673fcee4a60ec6af546b653f719455fce7f465de2e4ad43cab2118f8a1059bb3ff9d3a8272e 1
e29cfe5ee8028dc4e8126dbeda8707d691eb392aafb9d83134b22ffc99b7cc8c99ac0dc65a0a2ea9fa5c8e8d605283670fed38ffb2f46dd38e0bdd533e752d98 1
72d69359934a89e060bc3eab7bc4bf5355f02fff13f0b328fd0c9e6f46b11280fd099cbaf55f65fcda2

In [67]:
df_test.tail()

,client_id,path,sentence_id,gender,locale,clip,duration[ms]
595,71f0e83761e56f00349cd6c79d1006737d0cd343be0449...,common_voice_pl_20625708.mp3,3d79c296ba28132772fd9371b9fe9a1b937552d4b20f20...,male_masculine,pl,common_voice_pl_20625708.mp3,4704
596,71f0e83761e56f00349cd6c79d1006737d0cd343be0449...,common_voice_pl_20625710.mp3,2149056070ddf3c61a97fbc232d044f1c1b6202f613991...,male_masculine,pl,common_voice_pl_20625710.mp3,5736
597,71f0e83761e56f00349cd6c79d1006737d0cd343be0449...,common_voice_pl_20625715.mp3,45b8ca7f1f1ab168719837e103c4aabf2ad37e3b03facc...,male_masculine,pl,common_voice_pl_20625715.mp3,4224
598,71f0e83761e56f00349cd6c79d1006737d0cd343be0449...,common_voice_pl_20625871.mp3,43d23091c0a3bfa77d90480837a6dad3e6a1eb096d3144...,male_masculine,pl,common_voice_pl_20625871.mp3,4104
599,9dd724bc5710017cd3ece440a3faabf0cf95e03c098081...,common_voice_pl_20618336.mp3,5f9ddd4bd9f687a02025c21c9039f234a5a52485a0291f...,male_masculine,pl,common_voice_pl_20618336.mp3,7296


In [66]:
df_val

,client_id,path,sentence_id,gender,locale,clip,duration[ms]
0,9d59b50d1b7af635a9cd8c994413a46f6d7dc5d5d8c2b5...,common_voice_pl_20632912.mp3,cad984da353a3356b4fef356f62e50d08bfa1c6536a28d...,male_masculine,pl,common_voice_pl_20632912.mp3,6840
1,9d59b50d1b7af635a9cd8c994413a46f6d7dc5d5d8c2b5...,common_voice_pl_20632913.mp3,c6557b683ba04aad4b0e628d037416702b14df2e5163c2...,male_masculine,pl,common_voice_pl_20632913.mp3,5712
2,9d59b50d1b7af635a9cd8c994413a46f6d7dc5d5d8c2b5...,common_voice_pl_20632914.mp3,9ddbb92b42a03601d4fa82b89e8cf82fa8b59fe83d8fc9...,male_masculine,pl,common_voice_pl_20632914.mp3,8976
3,9d59b50d1b7af635a9cd8c994413a46f6d7dc5d5d8c2b5...,common_voice_pl_20632915.mp3,b51015ddc33ea68a0de43b973f40db626eab43ef9b5f1f...,male_masculine,pl,common_voice_pl_20632915.mp3,5784
4,9d59b50d1b7af635a9cd8c994413a46f6d7dc5d5d8c2b5...,common_voice_pl_20632916.mp3,c234583d74f4f2ad63f3abcdaa5f774b0f4a97e4409ca6...,male_masculine,pl,common_voice_pl_20632916.mp3,7872
...,...,...,...,...,...,...,...
595,5c35dba6992c970c13124e38db296603bcb5d7d02ebc6b...,common_voice_pl_20733240.mp3,4cf49be0f2c90bf71169529b30486b194f9b5b6b28e1dc...,male_masculine,pl,common_voice_pl_20733240.mp3,4320
596,5c35dba6992c970c13124e38db296603bcb5d7d02ebc6b...,common_voice_pl_20733242.mp3,3ffc379accec920518c5d208f39b3106a0a48d97afe670...,male_masculine,pl,common_voice_pl_20733242.mp3,7248
597,5c35dba6992c970c13124e38db296603bcb5d7d02ebc6b...,common_voice_pl_20733254.mp3,79b48504f54f9fc9cdc947b5cecb1825263e4c73c7ef0f...,male_masculine,pl,common_voice_pl_20733254.mp3,4152
598,5c35dba6992c970c13124e38db296603bcb5d7d02ebc6b...,common_voice_pl_20733256.mp3,6c89ca84c1bc698f532fe1363c72eadc2029a8b0199d5d...,male_masculine,pl,common_voice_pl_20733256.mp3,6408


In [68]:
df_train

,client_id,path,sentence_id,gender,locale,clip,duration[ms]
0,42eac75b48970bf7d7385bfd6560e5d25ba4b5a8940d8b...,common_voice_pl_20604557.mp3,4f6bb9cc663d47c1a3d2055960ea99109a1d37349cdbd0...,male_masculine,pl,common_voice_pl_20604557.mp3,5376
1,42eac75b48970bf7d7385bfd6560e5d25ba4b5a8940d8b...,common_voice_pl_20604606.mp3,5c5d0726da916a84e41c12b105470fae108a33378c69e7...,male_masculine,pl,common_voice_pl_20604606.mp3,4416
2,42eac75b48970bf7d7385bfd6560e5d25ba4b5a8940d8b...,common_voice_pl_20604667.mp3,5e422d14f50a12f23c74081de8bd6030cc984b4aadfaa4...,male_masculine,pl,common_voice_pl_20604667.mp3,4296
3,42eac75b48970bf7d7385bfd6560e5d25ba4b5a8940d8b...,common_voice_pl_20604688.mp3,521905b097b8899373fd7e2977b9dc13c07c2e181e6371...,male_masculine,pl,common_voice_pl_20604688.mp3,5496
4,42eac75b48970bf7d7385bfd6560e5d25ba4b5a8940d8b...,common_voice_pl_20604700.mp3,3235b5b3645f6edbf47c2504b44222f5c9962e037ae0c3...,male_masculine,pl,common_voice_pl_20604700.mp3,4416
...,...,...,...,...,...,...,...
1794,017dfbc927b7224d730d3bc892b8579a1ef07cf698a239...,common_voice_pl_20662065.mp3,590c0a675c40d4c9554d9d37cb2a9e9581663c888b5af6...,male_masculine,pl,common_voice_pl_20662065.mp3,4344
1795,017dfbc927b7224d730d3bc892b8579a1ef07cf698a239...,common_voice_pl_20662089.mp3,5754531b7e28d4581a1920f114cc5d7f471465d00a2958...,male_masculine,pl,common_voice_pl_20662089.mp3,5568
1796,017dfbc927b7224d730d3bc892b8579a1ef07cf698a239...,common_voice_pl_20662108.mp3,46b0e97d013757621ffc42d44bb258b2b02b5436f6ed50...,male_masculine,pl,common_voice_pl_20662108.mp3,4656
1797,017dfbc927b7224d730d3bc892b8579a1ef07cf698a239...,common_voice_pl_20662110.mp3,69077a01e8df0776fc5ed68b97e9fa98df15af3d1d8977...,male_masculine,pl,common_voice_pl_20662110.mp3,5928


In [69]:
df_train['client_id'].value_counts(ascending=False)

client_id
017dfbc927b7224d730d3bc892b8579a1ef07cf698a239f86f24241f49a4a7889da3f4abb34cf1f62b18513d2df573b0bac2574d649fa89d0fdca6141d31a008    19
df7558e6d831b0409d6693dd4ee6e6f3c90afa6f7f9862bf152d96daf58057baed97b8ab1b56685d7606065fddcae248d4369d84846bc4369e02bf88d9619033    19
260e58d8e72aad7c949a6271595ca587b88efe06321adf9756dc1736b4edef2c4ac01167b99d9dd1df2eb3c81b330bee58f04a1ba176b65b75197f0e9e6748d0    18
752862c870533d68643f57398eef6dd01c6dc98afb3576f5bfb3fa6c9be31a3ca003c4e4ad17d33e45bc06f4e8c9d71a16af8b1dd2732996603ee840cf180370    18
2bee416dbbb44825d407aa89597cd808caa95b92599ece2959d142ce4aed8eaaa5ba4461153561e94921b90ed1e4f03cff683d7da44a97bd7d84e773fc663398    18
                                                                                                                                    ..
18a8cec169bbe87e2e64c7e76b38aa83780d34aa1a92f77db2ea56cefc5cd54e3405f287274dff8155b1ae649186a332c088b7292a9a8215b099c3ceccb9204c     9
5bf680f0d5a57ec8f5fa676cc75a4955c03c7139fc90e

In [70]:
df_val['client_id'].value_counts(ascending=False)

client_id
5c35dba6992c970c13124e38db296603bcb5d7d02ebc6b5d36a0c1e12388e6956b4c1240ae8a5db1cb07d8986cf15e71e59c79a9dd9d74d3dee8fb50122ac3f8    9
efd9196732cda5e10e96839e1261e73b010fdbb65a6836a61f2db19da388a3e13b46ed915949244e7c6e1405d3892a8681b03319be59fa871d686c3bc5f79105    9
9aaf6c7732224697b1863a2263ecb8c194080f929a2ceba7d7d4e6a0a3973b6a64e897c4d5fbc56359c0181e4034d91e62242006b83c866daff77ff66258e1b8    9
549e8b7b914814f6be5d3051a444ea38ae2568fe095737177bf00110cc5751c632a7a52e575068daee69729e74de14ad2bf207f6fa98031009d680f34180739f    9
28970b9765499142a56a146eda2426ddb1dd4a0fc4035e3b1f20b4e212774b00eafe0f4e8276106a147e2d37afa13da85be8f346412112e76532c2a91b0988e6    9
                                                                                                                                   ..
048a345e57226655acbfa75d32a7db86f9503b0d12589683554acec5f43bce1bc92a9d6e277c5cf1b6867e38d03bb86bf8d64ff44442fa20e6d202c037b967ac    5
59735530ed153d5ce3a66ef9103f401a00b880b7febb191db372

In [71]:
df_test['client_id'].value_counts(ascending=False)

client_id
df2358cfdbed8e7cbf81230e2f4c696531c6f83a3ef9dfa1e7a8291623552ac9e3669154aef5960e9a17f22069e33db136e1206105b484a2b4cb2b807888286b    5
2e40d7e7e4af2830e773fee27eb0e7006e3692cac1ea5382e60e70d67cb1e91329b1d1efd79a58d9afcc70d295906832f7602bfeb4e53d5e997335101ce9f812    5
7176e3f3999023664f98ae52f7a27f89bee297f0ed9176406168705b34e64f58bac4b00a88e579877844eef061d9dbd625c7546bb919bbbfdbe65fdf28b6edb9    5
2bd74a1efa6728cbd69351fa2209c44504062556b904c4cbffc39a4e0d8cd6ad3eb9eb0057c2672197c9c549bbbe4d07bbbd53638b02f2a06f694479f2534c60    5
1a17b2f0d85fb13079adf9687b9788873d11fa472529e94f26a590c6afa99ef4cbf1448f4df29924e579a1c35644f76c38ad3c3d4ada543799e884a8eda11723    5
                                                                                                                                   ..
b3aa08530568be7ba965625222408790a7f169513ef5318e5907b8933609364022cc8befb5cb6ad0c6e373b68ae717fbb3f7327902a7365b44beab1efea7ab0e    1
7faf1b2a8074d32814fd8c78aaaea5abe0aa23ca45f183f5051a

## Audio Preprocessing

In [72]:
df_test_filenames = df_test['path']
df_val_filenames = df_val['path']
df_train_filenames = df_train['path']
df_test_filenames

0      common_voice_pl_20605185.mp3
1      common_voice_pl_21373330.mp3
2      common_voice_pl_23939982.mp3
3      common_voice_pl_20623923.mp3
4      common_voice_pl_20646602.mp3
                   ...             
595    common_voice_pl_20625708.mp3
596    common_voice_pl_20625710.mp3
597    common_voice_pl_20625715.mp3
598    common_voice_pl_20625871.mp3
599    common_voice_pl_20618336.mp3
Name: path, Length: 600, dtype: object

In [73]:
df_val_filenames

0      common_voice_pl_20632912.mp3
1      common_voice_pl_20632913.mp3
2      common_voice_pl_20632914.mp3
3      common_voice_pl_20632915.mp3
4      common_voice_pl_20632916.mp3
                   ...             
595    common_voice_pl_20733240.mp3
596    common_voice_pl_20733242.mp3
597    common_voice_pl_20733254.mp3
598    common_voice_pl_20733256.mp3
599    common_voice_pl_20733257.mp3
Name: path, Length: 600, dtype: object

In [74]:
PL_DIR = os.path.join('..', 'pl', 'clips')
PL_DIR

'..\\pl\\clips'

In [75]:
# TEST
df_train_filenames = df_train_filenames.apply(lambda fn: os.path.join(PL_DIR, fn))
df_val_filenames = df_val_filenames.apply(lambda fn: os.path.join(PL_DIR, fn))
df_test_filenames = df_test_filenames.apply(lambda fn: os.path.join(PL_DIR, fn))
df_val_filenames

0      ..\pl\clips\common_voice_pl_20632912.mp3
1      ..\pl\clips\common_voice_pl_20632913.mp3
2      ..\pl\clips\common_voice_pl_20632914.mp3
3      ..\pl\clips\common_voice_pl_20632915.mp3
4      ..\pl\clips\common_voice_pl_20632916.mp3
                         ...                   
595    ..\pl\clips\common_voice_pl_20733240.mp3
596    ..\pl\clips\common_voice_pl_20733242.mp3
597    ..\pl\clips\common_voice_pl_20733254.mp3
598    ..\pl\clips\common_voice_pl_20733256.mp3
599    ..\pl\clips\common_voice_pl_20733257.mp3
Name: path, Length: 600, dtype: object

In [76]:
# STARE
# pl_dataset_train = tf.data.Dataset.from_tensor_slices(df_train_filenames.apply(lambda fn: os.path.join(PL_DIR, fn)))
# pl_dataset_val = tf.data.Dataset.from_tensor_slices(df_val_filenames.apply(lambda fn: os.path.join(PL_DIR, fn)))
# pl_dataset_test = tf.data.Dataset.from_tensor_slices(df_test_filenames.apply(lambda fn: os.path.join(PL_DIR, fn)))



In [146]:
sample_rate = 48_000

def add_zeros(wav, sample_rate):
    time_probes = wav.shape[0]
    missing_probes_one_side = int((MIN_CLIP_DURATION/1000* sample_rate - time_probes)//2)
    padded_tensor = tf.pad(wav.numpy(), [[missing_probes_one_side, missing_probes_one_side]])
    return tf.convert_to_tensor(padded_tensor, dtype=tf.float32)

def cut_wav(wav, sample_rate):
    time_probes = wav.shape[0]
    # clip_dur_in_sec = time_probes / sample_rate
    overlap = int((time_probes - (MIN_CLIP_DURATION/1000) * sample_rate)/2) 
    cut_clip = wav[overlap:(time_probes - overlap)]
    return tf.convert_to_tensor(cut_clip, dtype=tf.float32)

def load_wav_16k_mono_and_resample(filename, fin_sam_rate=16_000):
    file_content = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(file_content, desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int32)
    # wav = librosa.resample(wav.numpy(), orig_sr=sample_rate.numpy(), target_sr=fin_sam_rate)
    # return wav
    # return tf.convert_to_tensor(wav, dtype=tf.float32), sample_rate.numpy()
    return tf.convert_to_tensor(wav, dtype=tf.float32)

def load_and_align_probes(file_path):
    wav = load_wav_16k_mono_and_resample(file_path)
    expected_probes = int((MIN_CLIP_DURATION/1000) * sample_rate)
    print(expected_probes)
    current_probes = wav.shape[0]
    print(current_probes)
    if expected_probes > current_probes:
        print("Add zeros")
        return add_zeros(wav, sample_rate)
    elif expected_probes < current_probes:
        print("Cut wav")
        return cut_wav(wav, sample_rate)
    return tf.convert_to_tensor(wav, dtype=tf.float32)



In [147]:
train_align_tensors = df_train_filenames.apply(lambda filename: load_and_align_probes(filename))
val_align_tensors = df_test_filenames.apply(lambda filename: load_and_align_probes(filename))
test_align_tensors = df_val_filenames.apply(lambda filename: load_and_align_probes(filename))

type(test_align_tensors)

192000
300672
Cut wav
192000
237312
Cut wav
192000
237312
Cut wav
192000
271872
Cut wav
192000
201600
Cut wav
192000
220032
Cut wav
192000
336384
Cut wav
192000
240768
Cut wav
192000
263808
Cut wav
192000
254592
Cut wav
192000
243072
Cut wav
192000
220032
Cut wav
192000
294912
Cut wav
192000
258048
Cut wav
192000
269568
Cut wav
192000
226944
Cut wav
192000
251136
Cut wav
192000
193536
Cut wav
192000
210816
Cut wav
192000
266112
Cut wav
192000
243072
Cut wav
192000
208512
Cut wav
192000
200448
Cut wav
192000
225792
Cut wav
192000
206208
Cut wav
192000
237312
Cut wav
192000
231552
Cut wav
192000
196992
Cut wav
192000
279936
Cut wav
192000
176256
Add zeros
192000
150912
Add zeros
192000
161280
Add zeros
192000
133632
Add zeros
192000
201600
Cut wav
192000
277632
Cut wav
192000
138240
Add zeros
192000
141696
Add zeros
192000
144000
Add zeros
192000
190080
Add zeros
192000
228096
Cut wav
192000
162432
Add zeros
192000
178560
Add zeros
192000
149760
Add zeros
192000
154368
Add zeros
192000
1

pandas.core.series.Series

In [148]:
test_align_tensors.to_list()

[<tf.Tensor: shape=(192000,), dtype=float32, numpy=
 array([ 0.00018311,  0.00018311,  0.00015259, ..., -0.03140259,
        -0.01834106, -0.00790405], dtype=float32)>,
 <tf.Tensor: shape=(192000,), dtype=float32, numpy=
 array([-0.0352478 , -0.02807617, -0.02102661, ..., -0.03704834,
        -0.03378296, -0.0300293 ], dtype=float32)>,
 <tf.Tensor: shape=(192000,), dtype=float32, numpy=
 array([-0.00030518, -0.00137329, -0.00216675, ...,  0.01394653,
         0.01254272,  0.01147461], dtype=float32)>,
 <tf.Tensor: shape=(192000,), dtype=float32, numpy=
 array([-0.00531006, -0.00506592, -0.00500488, ..., -0.00531006,
        -0.00549316, -0.00558472], dtype=float32)>,
 <tf.Tensor: shape=(192000,), dtype=float32, numpy=
 array([-0.00299072, -0.0027771 , -0.00299072, ...,  0.00204468,
         0.00213623,  0.00222778], dtype=float32)>,
 <tf.Tensor: shape=(192000,), dtype=float32, numpy=
 array([-0.00302124, -0.00271606, -0.00274658, ..., -0.00201416,
        -0.00210571, -0.00234985], dty

In [149]:
train_align_dataset = tf.data.Dataset.from_tensor_slices(train_align_tensors.to_list())
test_align_dataset = tf.data.Dataset.from_tensor_slices(val_align_tensors.to_list())
val_align_dataset = tf.data.Dataset.from_tensor_slices(test_align_tensors.to_list())

test_align_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(192000,), dtype=tf.float32, name=None)>

In [261]:
train_align_dataset.as_numpy_iterator().next()

array([6.1035156e-05, 6.1035156e-05, 3.0517578e-05, ..., 3.0517578e-05,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [196]:
def align_probes(wav, sample_rate):
    # wav = load_wav_16k_mono_and_resample(file_path)
    expected_probes = (MIN_CLIP_DURATION/1000) * sample_rate
    print(expected_probes)
    current_probes = wav.shape[0]
    print(current_probes)
    if expected_probes > current_probes:
        print("Add zeros")
        return add_zeros(wav, sample_rate)
    if expected_probes < current_probes:
        print("Cut wav")
        return cut_wav(wav, sample_rate)
    return wav
    

In [199]:
MIN_CLIP_DURATION/1000* sample_rate

192000.0

In [200]:
def increase_amplitude(wav, min_increase=2.0, max_increase=5.0):
    increased_wav = wav * random.uniform(min_increase, max_increase)
    return tf.convert_to_tensor(increased_wav, dtype=tf.float32)

In [203]:
def normalize_audio(wav):
    max_amplitude = tf.reduce_max(tf.abs(wav))
    normalized_wav = wav / max_amplitude  # Normalizacja do zakresu [-1, 1]
    return normalized_wav

In [206]:
def add_noise(wav, noise_level=0.1):
    noise = tf.random.normal(tf.shape(wav), mean=0.0, stddev=noise_level)
    return wav + noise

In [209]:
def time_masking(wav, max_mask_length=10000):
    # Sprawdzenie długości wav
        # Sprawdzenie czy wav jest pusty lub ma nieprawidłowy kształt
    if wav is None or tf.rank(wav) != 1:
        return wav
    
    # Sprawdzenie długości wav
    if tf.shape(wav)[0] <= max_mask_length:
        return wav
    
    # Losowa długość maskowania
    mask_length = tf.random.uniform([], maxval=max_mask_length, dtype=tf.int32)
    
    # Sprawdzenie, czy maska nie wyjdzie poza zakres
    mask_start_max = tf.shape(wav)[0] - mask_length
    mask_start = tf.random.uniform([], maxval=mask_start_max, dtype=tf.int32)
    
    # Stworzenie maski czasowej
    mask = tf.concat([
        tf.ones([mask_start]),
        tf.zeros([mask_length]),
        tf.ones([tf.shape(wav)[0] - mask_start - mask_length])
    ], axis=0)
    
    # Zastosowanie maskowania do sygnału audio
    masked_wav = wav * mask
    
    return masked_wav


In [212]:
def change_pitch(wav, sample_rate=48000, pitch_shift=2):
    wav_np = wav.numpy()  # Convert tensor to numpy array
    pitched_wav = librosa.effects.pitch_shift(wav_np, sr=sample_rate, n_steps=pitch_shift)
    return tf.convert_to_tensor(pitched_wav, dtype=tf.float32)


In [215]:
def speed_up_audio(wav, speed_factor=2):
    wav_np = wav.numpy()
    stretched_wav = librosa.effects.time_stretch(wav_np, rate=speed_factor)   
    return stretched_wav

In [218]:
def slow_down_audio(wav, speed_factor=0.5):
    wav_np = wav.numpy()
    slowed_wav_np = librosa.effects.time_stretch(wav_np, rate=speed_factor)
    return tf.convert_to_tensor(slowed_wav_np, dtype=tf.float32)


In [221]:

audio_processing_functions = [increase_amplitude, add_noise, time_masking, change_pitch]
def process_random_samples(dataset, num_samples_to_process):
    processed_samples = []
    for _ in range(num_samples_to_process):
        shuffled_dataset = dataset.shuffle(buffer_size=1000)
        random_sample =  next(iter(shuffled_dataset.take(1)))

        processing_function = random.choice(audio_processing_functions)
        processed_sample = processing_function(random_sample)

        processed_samples.append(processed_sample)

    return processed_samples

In [223]:
processed_samples_train = process_random_samples(train_align_dataset, TRAIN_SIZE - len(train_align_dataset))
processed_samples_val = process_random_samples(val_align_dataset, (VAL_SIZE - len(val_align_dataset)))
processed_samples_test = process_random_samples(test_align_dataset, (TEST_SIZE - len(test_align_dataset)))
processed_samples_test

In [224]:
align_processed_train = [align_probes(tensor, sample_rate) for tensor in processed_samples_train]
align_processed_val = [align_probes(tensor, sample_rate) for tensor in processed_samples_val]
align_processed_test = [align_probes(tensor,sample_rate) for tensor in processed_samples_test]
align_processed_train

192000.0
192000
192000.0
192000
192000.0
192000
192000.0
192000
192000.0
192000
192000.0
192000


[<tf.Tensor: shape=(192000,), dtype=float32, numpy=
 array([-0.00444812, -0.00512552,  0.11092313, ...,  0.13542885,
         0.10090639,  0.20667386], dtype=float32)>,
 <tf.Tensor: shape=(192000,), dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>]

In [225]:
processed_samples_train_dataset = tf.data.Dataset.from_tensor_slices(align_processed_train)
processed_samples_val_dataset = tf.data.Dataset.from_tensor_slices(align_processed_val)
processed_samples_test_dataset = tf.data.Dataset.from_tensor_slices(align_processed_test)

In [226]:
processed_samples_test_dataset.as_numpy_iterator()

NumpyIterator(iterator=<tensorflow.python.data.ops.iterator_ops.OwnedIterator object at 0x00000197836A71C0>)

In [227]:

train_dataset_con = train_align_dataset.concatenate(processed_samples_train_dataset)
val_dataset_con = val_align_dataset.concatenate(processed_samples_val_dataset)
test_dataset_con = test_align_dataset.concatenate(processed_samples_test_dataset)


In [228]:
align_pl_dataset_train_with_processed_samples_norm = train_dataset_con.map(lambda audio: normalize_audio(audio))
align_pl_dataset_val_with_processed_samples_norm = val_dataset_con.map(lambda audio: normalize_audio(audio))
align_pl_dataset_test_with_processed_samples_norm = test_dataset_con.map(lambda audio: normalize_audio(audio))

In [229]:
def create_spectrogram(wav):
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

In [232]:
spectrogram_pl_dataset_train = align_pl_dataset_train_with_processed_samples_norm.map(lambda audio: create_spectrogram(audio))
spectrogram_pl_dataset_val = align_pl_dataset_val_with_processed_samples_norm.map(lambda audio: create_spectrogram(audio))
spectrogram_pl_dataset_test = align_pl_dataset_test_with_processed_samples_norm.map(lambda audio: create_spectrogram(audio))

In [233]:
spectrogram_pl_dataset_train

<_MapDataset element_spec=TensorSpec(shape=(5991, 257, 1), dtype=tf.float32, name=None)>

In [238]:
spectrogram_pl_dataset_val

<_MapDataset element_spec=TensorSpec(shape=(5991, 257, 1), dtype=tf.float32, name=None)>

In [235]:
spectrogram_pl_dataset_test

<_MapDataset element_spec=TensorSpec(shape=(5991, 257, 1), dtype=tf.float32, name=None)>

In [243]:
spectrogram_pl_dataset_test.as_numpy_iterator().next().shape

(5991, 257, 1)

In [257]:
labeled_train_dataset = tf.data.Dataset.zip((spectrogram_pl_dataset_train, tf.data.Dataset.from_tensor_slices(tf.ones(len(spectrogram_pl_dataset_train)))))
labeled_val_dataset = tf.data.Dataset.zip((spectrogram_pl_dataset_val, tf.data.Dataset.from_tensor_slices(tf.ones(len(spectrogram_pl_dataset_val)))))
labeled_test_dataset = tf.data.Dataset.zip((spectrogram_pl_dataset_test, tf.data.Dataset.from_tensor_slices(tf.ones(len(spectrogram_pl_dataset_test)))))

In [258]:
labeled_train_dataset.as_numpy_iterator().next()

(array([[[1.8458270e-02],
         [1.6165592e-02],
         [1.0292490e-02],
         ...,
         [8.6339860e-05],
         [6.4029133e-05],
         [8.6291693e-05]],
 
        [[1.3577014e-02],
         [1.2670887e-02],
         [8.7124947e-03],
         ...,
         [1.4070264e-04],
         [1.5792389e-04],
         [1.6248692e-04]],
 
        [[7.2781579e-03],
         [9.0379585e-03],
         [7.6832026e-03],
         ...,
         [2.0666419e-04],
         [2.5137153e-04],
         [2.6400248e-04]],
 
        ...,
 
        [[4.2442861e-03],
         [5.3159436e-03],
         [4.9108407e-03],
         ...,
         [3.0186138e-05],
         [4.9599432e-05],
         [4.6078581e-05]],
 
        [[7.5082842e-04],
         [4.3893629e-03],
         [4.9649389e-03],
         ...,
         [5.2364692e-05],
         [4.3099248e-05],
         [1.3067300e-05]],
 
        [[2.8126733e-03],
         [4.7225589e-03],
         [4.9914052e-03],
         ...,
         [7.1507347e-05],
  

In [259]:
labeled_val_dataset.as_numpy_iterator().next()

(array([[[4.83654952e-03],
         [6.44301251e-03],
         [7.39035383e-03],
         ...,
         [1.14201277e-04],
         [1.22252779e-04],
         [1.07528176e-04]],
 
        [[5.59206493e-03],
         [6.35993434e-03],
         [6.71966700e-03],
         ...,
         [7.76600209e-05],
         [8.38616179e-05],
         [3.90484929e-05]],
 
        [[3.60069401e-03],
         [5.55087859e-03],
         [6.52545970e-03],
         ...,
         [6.10134048e-05],
         [6.44259126e-05],
         [2.95548234e-05]],
 
        ...,
 
        [[1.32053718e-03],
         [8.67330749e-03],
         [1.31999077e-02],
         ...,
         [1.00748672e-04],
         [6.17914193e-05],
         [2.37636268e-05]],
 
        [[5.99709153e-03],
         [8.95410497e-03],
         [1.19269453e-02],
         ...,
         [8.72119563e-05],
         [6.04719462e-05],
         [2.88337469e-05]],
 
        [[3.52966413e-03],
         [1.06917219e-02],
         [1.30438292e-02],
         

In [260]:
labeled_test_dataset.as_numpy_iterator().next()

(array([[[2.1117203e+01],
         [1.9624756e+01],
         [1.4369721e+01],
         ...,
         [8.7114451e-03],
         [1.0419948e-02],
         [1.1101723e-02]],
 
        [[1.3749129e+01],
         [1.6874081e+01],
         [1.6693989e+01],
         ...,
         [6.0858387e-03],
         [8.0109444e-03],
         [9.0990067e-03]],
 
        [[3.1875930e+00],
         [1.4700436e+01],
         [1.7415436e+01],
         ...,
         [3.1180547e-03],
         [4.9122497e-03],
         [6.0904026e-03]],
 
        ...,
 
        [[1.5304584e+01],
         [1.6130989e+01],
         [1.4567617e+01],
         ...,
         [5.0403532e-03],
         [3.6468508e-03],
         [7.3528290e-04]],
 
        [[2.1554758e+01],
         [1.9867685e+01],
         [1.4708412e+01],
         ...,
         [6.5290965e-03],
         [5.2083633e-03],
         [2.2735596e-03]],
 
        [[2.5520439e+01],
         [2.1361755e+01],
         [1.2366341e+01],
         ...,
         [8.1429426e-03],
  